Створити RNN для семантичного аналізу тексту з використанням: моделі LSTM Розробити власну архітектуру, кількість рекурентних шарів, функції активації, вибрати оптимізатор. Отримати для двох моделей по дві рецензії (позитивну і негативну).

In [1]:
import numpy as np
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

positive_reviews = [
    "Яскраве сонечко, безхмарне небо і приємна прохолода - ідеальна комбінація для відпочинку на свіжому повітрі."]

negative_reviews = [
    "Несподівані дощі змочили всі плани на вихідні! Сильний вітер розкидав усю метушню по вулицях."]

# Об'єднання рецензій
reviews = positive_reviews + negative_reviews
count_positive = len(positive_reviews)
count_negative = len(negative_reviews)
total_reviews = count_positive + count_negative

# Токенізація тексту
max_words_count = 500
max_text_len = 10
tokenizer = Tokenizer(num_words=max_words_count, filters='!–"—#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(reviews)

# Перетворення тексту в числову послідовність 
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(reviews[0][:100])

data = tokenizer.texts_to_sequences(reviews)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(list(tokenizer.word_index.items()))
# Створення вхідних та вихідних даних для моделі
X = data_pad
Y = np.array([1] * count_positive + [0] * count_negative)

# Випадкове перемішування даних
indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

# Побудова моделі LSTM
model = Sequential()
model.add(Embedding(max_words_count, 128, input_length=max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Компіляція та тренування моделі
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=30)





[('яскраве', 1), ('сонечко', 1), ('безхмарне', 1), ('небо', 1), ('і', 1), ('приємна', 1), ('прохолода', 1), ('ідеальна', 1), ('комбінація', 1), ('для', 1)]
Яскраве сонечко, безхмарне небо і приємна прохолода - ідеальна комбінація для відпочинку на свіжому 
[('на', 1), ('яскраве', 2), ('сонечко', 3), ('безхмарне', 4), ('небо', 5), ('і', 6), ('приємна', 7), ('прохолода', 8), ('ідеальна', 9), ('комбінація', 10), ('для', 11), ('відпочинку', 12), ('свіжому', 13), ('повітрі', 14), ('несподівані', 15), ('дощі', 16), ('змочили', 17), ('всі', 18), ('плани', 19), ('вихідні', 20), ('сильний', 21), ('вітер', 22), ('розкидав', 23), ('усю', 24), ('метушню', 25), ('по', 26), ('вулицях', 27)]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 128)           64000     
                                                                 
 lstm (LSTM)                 (

In [5]:
# Перевірка нової рецензії
test_review = "Сьогодні на вулиці прекрасна погода! Чудовий сонячний день."
test_data = tokenizer.texts_to_sequences([test_review])
test_data_pad = pad_sequences(test_data, maxlen=max_text_len)

# Прогнозування класу нової рецензії
prediction = model.predict(test_data_pad)
if prediction > 0.5:
    print("Результат: Позитивна рецензія")
else:
    print("Результат: Негативна рецензія")



1/1 [==============================] - 0s 34ms/step
Результат: Позитивна рецензія


In [6]:
# Перевірка нової рецензії
test_review_1 = "Сірі жахливі хмари, що застилають насуплене небо.".lower()
test_data_1 = tokenizer.texts_to_sequences([test_review_1])
test_data_pad_1 = pad_sequences(test_data_1, maxlen=max_text_len)

# Прогнозування класу нової рецензії
prediction = model.predict(test_data_pad_1)
if prediction > 0.5:
    print("Результат: Позитивна рецензія")
else:
    print("Результат: Негативна рецензія")

1/1 [==============================] - 0s 34ms/step
Результат: Негативна рецензія
